# Importações


In [258]:
import geopandas
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import random

# Configurações iniciais


In [259]:
load_dotenv()

True

# Leitura dos dados


In [260]:
inventory: pd.DataFrame = pd.read_csv(
    os.environ.get("INVENTORY_FILE_PATH"),
    encoding="ISO-8859-1",
)

In [261]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313


In [262]:
inventory_plot_location: geopandas.GeoDataFrame = geopandas.read_file(
    os.environ.get("INVENTORY_PLOT_LOCATION_FILE_PATH"),
)

In [263]:
inventory_plot_location.head()

,plot_ID,geometry
0,P20,"POLYGON ((840040.725 9674392.229, 840082.852 9..."
1,P02,"POLYGON ((840077.054 9674170.361, 840124.283 9..."
2,P07,"POLYGON ((840331.289 9673499.875, 840372.258 9..."
3,P09,"POLYGON ((840301.102 9673285.583, 840340.905 9..."
4,P08,"POLYGON ((839205.643 9673316.074, 839256.495 9..."


In [264]:
wood_density_dataframe = pd.read_excel(
    os.environ.get("WOOD_DENSITY_FILE_PATH"),
    sheet_name="Data",
)

In [265]:
wood_density_dataframe.head()

,Number,Family,Binomial,"Wood density (g/cm^3), oven dry mass/fresh volume",Region,Reference Number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


# Exploração dos dados


In [266]:
inventory["scientific.name"].unique(), len(inventory["scientific.name"].unique())

(array(['Licania guianensis', 'Sacoglottis guianensis', 'Pourouma minor',
        'Pouteria gongrijpii', 'Peltogyne lecointei', nan,
        'Protium paniculatum', 'Qualea paraensis', 'Mouriri collocarpa',
        'Eschweilera ovata', 'Pouteria sp.', 'Ormosia grossa',
        'Ocotea cernua', 'Eperua bijuga', 'Brosimum rubescens',
        'Oenocarpus bataua', 'Hevea brasiliensis', 'Inga thibaudiana',
        'Eriotheca globosa', 'Iryanthera paraensis', 'Dipteryx odorata',
        'Sterculia pruriens', 'Miconia poeppigii', 'Sloanea guianensis',
        'Poeppigia procera', 'Xylopia sp.', 'Neea oppositifolia',
        'Geissospermum sericeum', 'Goupia glabra', 'Eschweilera micrantha',
        'Inga alba', 'Diospyros artanthifolia', 'Eschweilera coriacea',
        'Micrandra minor', 'Trichilia micrantha',
        'Tetragastris panamensis', 'Ormosia coccinea', 'Guarea guidonia',
        'Couepia robusta', 'Pouteria guianensis', 'Ocotea canaliculata',
        'Licania octandra', 'Guatteria 

In [267]:
inventory["family.name"].unique(), len(inventory["family.name"].unique())

(array(['Chrysobalanaceae', 'Humiriaceae', 'Urticaceae', 'Sapotaceae',
        'Fabaceae', nan, 'Burseraceae', 'Vochysiaceae', 'Melastomataceae',
        'Lecythidaceae', 'Lauraceae', 'Moraceae', 'Arecaceae',
        'Euphorbiaceae', 'Malvaceae', 'Myristicaceae', 'Elaeocarpaceae',
        'Annonaceae', 'Nyctaginaceae', 'Apocynaceae', 'Goupiaceae',
        'Ebenaceae', 'Meliaceae', 'Violaceae', 'Olacaceae',
        'Caryocaraceae', 'Combretaceae', 'Loganiaceae', 'Myrtaceae',
        'Simaroubaceae', 'Rubiaceae', 'Bignoniaceae', 'Malpighiaceae',
        'Salicaceae', 'Boraginaceae', 'Anacardiaceae', 'Lamiaceae',
        'Ochnaceae', 'Clusiaceae', 'Calophyllaceae'], dtype=object),
 40)

# Transformações


## Wood Density

In [268]:
wood_density_dataframe = wood_density_dataframe.rename(
    {
        "Family": "family.name",
        "Binomial": "scientific.name",
        "Wood density (g/cm^3), oven dry mass/fresh volume": "wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

In [269]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name", "scientific.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [270]:
wood_density_dataframe.head()

,Number,family.name,scientific.name,wood.density,Region,reference.number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [271]:
inventory = inventory.merge(
    mean_wood_density_dataframe,
    on="scientific.name",
    how="left",
)

In [272]:
inventory = inventory.rename(
    {
        "family.name_x": "family.name",
        "Binomial": "scientific.name",
        "wood.density": "family.scientific.wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

In [273]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,family.scientific.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333


In [274]:
mean_wood_density_dataframe = (
    wood_density_dataframe.groupby(["family.name"])["wood.density"]
    .mean("wood.density")
    .reset_index()
)

In [275]:
mean_wood_density_dataframe = mean_wood_density_dataframe.rename(
    {
        "wood.density": "family.wood.density",
    },
    axis=1,
)

mean_wood_density_dataframe.head()

,family.name,family.wood.density
0,Acanthaceae,0.670709
1,Achariaceae,0.605300
2,Actinidiaceae,0.398541
3,Adoxaceae,0.446333
4,Akaniaceae,0.560710


In [276]:
inventory = inventory.merge(mean_wood_density_dataframe, on="family.name", how="left")


# wood_density_dataframe["reference.number"].apply(str)

In [277]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.name_y,family.scientific.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,Humiriaceae,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,Urticaceae,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,Sapotaceae,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,Fabaceae,0.763333,0.678134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,Vochysiaceae,0.523047,0.551651
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,Fabaceae,0.585129,0.678134
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,Malvaceae,0.410000,0.476090
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,NaN,0.697848


In [278]:
inventory = inventory.drop("family.name_y", axis=1)

In [279]:
inventory = inventory.rename(
    {"family.name_x": "family.name", "scientific.name_x": "scientific.name"},
    axis=1,
)

In [280]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134


In [281]:
inventory["wood.density"] = inventory["family.scientific.wood.density"].combine_first(
    inventory["family.wood.density"]
)

In [282]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,0.551651,0.523047
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,0.678134,0.585129
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,0.476090,0.410000
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,0.697848,0.697848


In [283]:
mean_wood_density = inventory[inventory["wood.density"].notnull()][
    "wood.density"
].mean()

In [284]:
inventory = inventory.fillna(value={"wood.density": mean_wood_density})

## Tree height

In [285]:
inventory["tree.height"] = random.choice([38.1 + 1.8, 38.1 + 1.8]) * (
    1
    - np.exp(
        random.choice([-0.0693 + 0.0044, -0.0693 - 0.0044])
        * inventory["DBH"] ** random.choice([0.826 + 0.040, 0.826 - 0.040])
    )
)

In [286]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701,33.978553
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775,32.656266
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953,28.890326
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403,31.482594
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333,15.128921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,20160830,840072.5767,9674390.969,0.523047,0.551651,0.523047,25.475342
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,20160830,840071.0245,9674395.507,0.585129,0.678134,0.585129,18.818011
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,20160830,840069.0191,9674396.896,0.410000,0.476090,0.410000,24.369783
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,20160830,840069.1150,9674398.094,NaN,0.697848,0.697848,16.976239


In [287]:
inventory = inventory.merge(
    inventory.groupby("plot")["tree.height"]
    .mean()
    .reset_index()
    .rename({"tree.height": "lorey.height"}, axis=1),
    on="plot",
)

## Snag density

In [288]:
inventory["snag.density"] = (
    (31 * 0.71)
    + (26 * 0.69)
    + (24 * 0.66)
    + (18 * 0.59)
    + (18 * 0.33)
    + (113 * 0.52)
    + (43 * 0.5)
    + (88 * 0.6)
    + (35 * 0.7)
    + (48 * 0.58)
    + (52 * 0.45)
    + (21 * 0.34)
    + (103 * 0.36)
    + (86 * 0.45)
) / (31 + 26 + 24 + 18 + 18 + 113 + 43 + 88 + 35 + 48 + 52 + 21 + 103 + 86)

In [289]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,0.780701,0.780701,33.978553,25.374408,0.51568
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.835775,0.771500,0.835775,32.656266,25.374408,0.51568
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705,0.435953,0.344090,0.435953,28.890326,25.374408,0.51568
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803,0.799403,0.702085,0.799403,31.482594,25.374408,0.51568
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313,0.763333,0.678134,0.763333,15.128921,25.374408,0.51568


## Tree vs Palms vs Lianas

In [290]:
inventory["palm"] = np.where(inventory["family.name"].str.contains("Arecaceae"), 1, 0)
inventory["palm"] = np.where(
    inventory["family.name"].str.contains("Palmae"), 1, inventory["palm"]
)
inventory["palm"] = np.where(
    inventory["family.name"].str.contains("Palmaceae"), 1, inventory["palm"]
)

inventory.palm.unique()

array([0, 1])

In [291]:
inventory["palm"] = np.where(
    (inventory["family.name"].str.contains("Arecaceae"))
    | (inventory["family.name"].str.contains("Palmae"))
    | (inventory["family.name"].str.contains("Palmaceae")),
    1,
    0,
)

In [292]:
inventory[inventory["palm"] == 1]["palm"].count()

55

In [293]:
inventory["liana"] = np.where(
    inventory["common.name"].str.contains("liana"),
    1,
    0,
)

In [294]:
inventory[inventory["liana"] == 1]["palm"].count()

7

In [295]:
inventory[
    (inventory["family.wood.density"].isna())
    & (inventory["family.scientific.wood.density"].isna())
]["common.name"].unique()

array(['morta', 'tamaguare', 'ucuuba da varzea', 'carana', 'fava rosa',
       'pincel de macaco', 'breu brrote', 'abacabeira', 'jarana vermelha',
       'taguari', 'tanenbuco amarelo', 'acaizeiro', 'liana',
       'faveira amargosa', 'zaguari', 'virola/ucuuba preta',
       'muiracatiara rajada', 'abiu rosadinha', 'burra leteira',
       'abirana', 'gaivota', 'abirana branca', 'carape', 'mirindiba',
       'pedra ume', 'brteu branco', 'timbora', 'mamominho',
       'ucuuba folha grande', 'tanenbuco', 'faveira rosa',
       'nao identificada', 'guajara vermelho', 'muiracacaco',
       'acariguara', 'acariguarana', 'puriu', ' liana', 'seringuarana',
       'nao indetificada', 'manguirana', 'virola(ucuuba)', 'sorvao',
       'urucurana branca', 'tento cachua', 'ipê amarelo', 'uxi coroa',
       'quariuba', 'mirapiranga', 'ucuuba t.f', 'envira cunacaru',
       'seringarana preta', 'pocoro', 'cacaui', 'urucurana branco'],
      dtype=object)

In [296]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,...,UTM.Easting,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density,palm,liana
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,...,840127.0705,9674180.998,NaN,0.780701,0.780701,33.978553,25.374408,0.51568,0,0
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,...,840098.9678,9674177.868,0.835775,0.771500,0.835775,32.656266,25.374408,0.51568,0,0
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,...,840126.4377,9674180.705,0.435953,0.344090,0.435953,28.890326,25.374408,0.51568,0,0
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,...,840109.4829,9674171.803,0.799403,0.702085,0.799403,31.482594,25.374408,0.51568,0,0
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,...,840125.6847,9674181.313,0.763333,0.678134,0.763333,15.128921,25.374408,0.51568,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,...,840072.5767,9674390.969,0.523047,0.551651,0.523047,25.475342,23.634710,0.51568,0,0
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,...,840071.0245,9674395.507,0.585129,0.678134,0.585129,18.818011,23.634710,0.51568,0,0
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,...,840069.0191,9674396.896,0.410000,0.476090,0.410000,24.369783,23.634710,0.51568,0,0
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,...,840069.1150,9674398.094,NaN,0.697848,0.697848,16.976239,23.634710,0.51568,0,0


## ACD

In [297]:
fc = 0.5

living_trees_eq = (
    0.0673
    * fc
    * (inventory["wood.density"] * (inventory["DBH"] ** 2) * inventory["tree.height"])
    ** 0.976
)

living_palms_eq = 0.03781 * fc * (inventory["DBH"] ** 2.7483)

living_lianas_eq = 0.03798 * fc * (inventory["DBH"] ** 2.657)

dead_trees_eq = (
    0.1007
    * fc
    * inventory["snag.density"]
    * (inventory["DBH"] ** 2)
    * (inventory["tree.height"] ** 0.818)
)

In [298]:
inventory["IACD"] = np.where(
    (inventory["palm"] == 0) & (inventory["liana"] == 0) & (inventory["Dead"] == False),
    living_trees_eq,
    np.where(
        (inventory["palm"] == 1)
        & (inventory["liana"] == 0)
        & (inventory["Dead"] == False),
        living_palms_eq,
        np.where(
            (inventory["palm"] == 0)
            & (inventory["liana"] == 1)
            & (inventory["Dead"] == False),
            living_lianas_eq,
            dead_trees_eq,
        ),
    ),
)

In [299]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,...,UTM.Northing,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density,palm,liana,IACD
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,...,9674180.998,NaN,0.780701,0.780701,33.978553,25.374408,0.51568,0,0,1683.009746
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,...,9674177.868,0.835775,0.771500,0.835775,32.656266,25.374408,0.51568,0,0,1345.406058
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,...,9674180.705,0.435953,0.344090,0.435953,28.890326,25.374408,0.51568,0,0,335.331916
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,...,9674171.803,0.799403,0.702085,0.799403,31.482594,25.374408,0.51568,0,0,1009.940170
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,...,9674181.313,0.763333,0.678134,0.763333,15.128921,25.374408,0.51568,0,0,32.810036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,DUC_A01,20,048a,quarubarana,Erisma uncinatum,Vochysiaceae,24.0,O,False,NaN,...,9674390.969,0.523047,0.551651,0.523047,25.475342,23.634710,0.51568,0,0,208.365650
1165,DUC_A01,20,049a,fava saboeiro,Abarema jupunba,Fabaceae,14.0,O,False,NaN,...,9674395.507,0.585129,0.678134,0.585129,18.818011,23.634710,0.51568,0,0,60.401568
1166,DUC_A01,20,050a,mamorana,Eriotheca globosa,Malvaceae,22.0,O,False,NaN,...,9674396.896,0.410000,0.476090,0.410000,24.369783,23.634710,0.51568,0,0,132.751186
1167,DUC_A01,20,051a,louro cheiroso,Diospyros artanthifolia,Ebenaceae,11.9,O,False,NaN,...,9674398.094,NaN,0.697848,0.697848,16.976239,23.634710,0.51568,0,0,47.237709


In [300]:
inventory.groupby("plot")["IACD"].sum()

plot
2     31280.732149
4     36500.455423
5     25155.888096
7     47279.437251
8     34716.783619
9     35449.316078
10    28308.854374
11    28561.378971
12    41076.039113
13    32846.642791
14    31202.155373
15    26922.722415
16    18825.178200
17    25215.886482
18    33328.577424
19    34838.747308
20    25363.425974
Name: IACD, dtype: float64

## Área do plot

In [301]:
inventory_plot_location

,plot_ID,geometry
0,P20,"POLYGON ((840040.725 9674392.229, 840082.852 9..."
1,P02,"POLYGON ((840077.054 9674170.361, 840124.283 9..."
2,P07,"POLYGON ((840331.289 9673499.875, 840372.258 9..."
3,P09,"POLYGON ((840301.102 9673285.583, 840340.905 9..."
4,P08,"POLYGON ((839205.643 9673316.074, 839256.495 9..."
5,P11,"POLYGON ((839297.741 9673134.604, 839339.666 9..."
6,P10,"POLYGON ((841354.435 9673642.624, 841401.045 9..."
7,P12,"POLYGON ((841393.219 9673456.688, 841436.904 9..."
8,P04,"POLYGON ((841168.997 9674649.126, 841205.644 9..."
9,P05,"POLYGON ((841201.669 9674452.071, 841249.937 9..."


In [302]:
tost = inventory_plot_location.copy()
tost = tost.to_crs({"init": "epsg:3857"})
tost = tost.rename({"plot_ID": "plot"})

c:\Users\joaov\Documents\UFMG\TCC\Repository\canopyEstimation\venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [303]:
tost["area.km2"] = tost["geometry"].area / 10**6

tost

,plot_ID,geometry,area.km2
0,P20,"POLYGON ((-6672667.325 -327605.032, -6672625.2...",0.001953
1,P02,"POLYGON ((-6672630.378 -327828.353, -6672583.1...",0.002508
2,P07,"POLYGON ((-6672374.229 -328502.830, -6672333.3...",0.002115
3,P09,"POLYGON ((-6672403.834 -328718.707, -6672364.0...",0.002194
4,P08,"POLYGON ((-6673499.690 -328691.035, -6673448.8...",0.002607
5,P11,"POLYGON ((-6673407.067 -328873.523, -6673365.1...",0.002480
6,P10,"POLYGON ((-6671351.195 -328356.243, -6671304.6...",0.002614
7,P12,"POLYGON ((-6671311.887 -328543.371, -6671268.2...",0.002565
8,P04,"POLYGON ((-6671539.455 -327343.220, -6671502.8...",0.002136
9,P05,"POLYGON ((-6671506.232 -327541.562, -6671458.0...",0.002548


In [304]:
tost = pd.DataFrame(tost.rename({"plot_ID": "plot"}, axis=1))
tost["plot"] = tost["plot"].str.replace(r"(P|0)", r"", case=True, regex=True)
tost["plot"] = pd.to_numeric(tost["plot"], errors="coerce").astype(np.int64)
tost

,plot,geometry,area.km2
0,2,"POLYGON ((-6672667.325 -327605.032, -6672625.2...",0.001953
1,2,"POLYGON ((-6672630.378 -327828.353, -6672583.1...",0.002508
2,7,"POLYGON ((-6672374.229 -328502.830, -6672333.3...",0.002115
3,9,"POLYGON ((-6672403.834 -328718.707, -6672364.0...",0.002194
4,8,"POLYGON ((-6673499.690 -328691.035, -6673448.8...",0.002607
5,11,"POLYGON ((-6673407.067 -328873.523, -6673365.1...",0.002480
6,1,"POLYGON ((-6671351.195 -328356.243, -6671304.6...",0.002614
7,12,"POLYGON ((-6671311.887 -328543.371, -6671268.2...",0.002565
8,4,"POLYGON ((-6671539.455 -327343.220, -6671502.8...",0.002136
9,5,"POLYGON ((-6671506.232 -327541.562, -6671458.0...",0.002548


In [305]:
inventory = inventory.merge(tost[["area.km2", "plot"]], on="plot")

In [306]:
inventory

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,...,family.scientific.wood.density,family.wood.density,wood.density,tree.height,lorey.height,snag.density,palm,liana,IACD,area.km2
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,...,NaN,0.780701,0.780701,33.978553,25.374408,0.51568,0,0,1683.009746,0.001953
1,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,...,NaN,0.780701,0.780701,33.978553,25.374408,0.51568,0,0,1683.009746,0.002508
2,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,...,0.835775,0.771500,0.835775,32.656266,25.374408,0.51568,0,0,1345.406058,0.001953
3,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,...,0.835775,0.771500,0.835775,32.656266,25.374408,0.51568,0,0,1345.406058,0.002508
4,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,...,0.435953,0.344090,0.435953,28.890326,25.374408,0.51568,0,0,335.331916,0.001953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,DUC_A01,19,067a,uxirana,Sacoglottis guianensis,Humiriaceae,17.5,O,False,NaN,...,0.835775,0.771500,0.835775,21.498698,23.513369,0.51568,0,0,150.587511,0.002181
1071,DUC_A01,19,068a,matamata preto,Eschweilera ovata,Lecythidaceae,11.7,O,False,NaN,...,0.900000,0.696321,0.900000,16.790372,23.513369,0.51568,0,0,57.954003,0.002181
1072,DUC_A01,19,069a,muiratinga,Maquira coriacea,Moraceae,27.8,O,False,NaN,...,0.511200,0.541326,0.511200,27.336074,23.513369,0.51568,0,0,290.803518,0.002181
1073,DUC_A01,19,070a,liana,NaN,NaN,20.7,L,False,NaN,...,NaN,NaN,0.665075,23.598303,23.513369,0.51568,0,1,59.574194,0.002181


In [311]:
inventory["area.m2"] = inventory["area.km2"] * 1000000

## ACD

In [312]:
inventory.groupby("plot").sum("IACD")["IACD"] / inventory.groupby("plot").sum(
    "area.m2"
)["area.m2"]

plot
2     0.259718
4     0.221882
5     0.164560
7     0.275987
8     0.201758
9     0.209830
11    0.185719
12    0.235484
13    0.221099
14    0.190126
15    0.231988
16    0.158400
17    0.200635
18    0.144975
19    0.192420
dtype: float64